In [ ]:
!sudo apt-get update && sudo apt-get install -y apt-transport-https
!curl -s https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!echo "deb https://apt.kubernetes.io/ kubernetes-xenial main" | sudo tee -a /etc/apt/sources.list.d/kubernetes.list
!sudo apt-get update
!sudo apt-get install -y kubectl

In [ ]:
!curl https://raw.githubusercontent.com/kubernetes/helm/master/scripts/get | bash

In [ ]:
!sudo apt-get install gir1.2-glib-2.0

In [ ]:
!sudo apt-get upgrade -y

In [ ]:
!pip install --upgrade dask[complete] dask-kubernetes dask-gateway 

**RESTART YOUR KERNEL**

In [ ]:
!az login >> /dev/null

In [ ]:
!az account show

In [ ]:
import os
os.environ['RG']  = 'cody-dask-aks-rg'
os.environ['AKS'] = 'dask-aks'
os.environ['LOC'] = 'westus2'
os.environ['VM']  = 'STANDARD_DS12_V2'
os.environ['MIN'] = '3'
os.environ['MAX'] = '100'

In [ ]:
!az group create -n $RG -l $LOC

In [ ]:
!az aks create -n $AKS -g $RG --generate-ssh-keys --enable-cluster-autoscaler --min-count $MIN --max-count $MAX

In [ ]:
!az aks show -n $AKS -g $RG

In [ ]:
!az aks get-credentials -n $AKS -g $RG --overwrite-existing

In [ ]:
!kubectl --namespace kube-system create serviceaccount tiller

In [ ]:
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller

In [ ]:
!helm init --service-account tiller --history-max 100 --wait

In [ ]:
!kubectl patch deployment tiller-deploy --namespace=kube-system --type=json --patch='[{"op": "add", "path": "/spec/template/spec/containers/0/command", "value": ["/tiller", "--listen=localhost:44134"]}]'

In [ ]:
!helm version

In [ ]:
from dask_kubernetes import KubeCluster, make_pod_spec

In [ ]:
pod_spec = make_pod_spec(image='daskdev/dask:latest',
                          memory_limit='1G', memory_request='1G',
                          cpu_limit=1, cpu_request=1,
                          env={'EXTRA_PIP_PACKAGES': 'dask-lightgbm'}
                        ) 

In [ ]:
cluster = KubeCluster(pod_spec)

In [ ]:
cluster

In [ ]:
cluster.scale(10)

In [ ]:
from dask.distributed import Client

In [ ]:
c = Client(cluster)

In [ ]:
c

In [ ]:
c.get_versions(check=True)

In [ ]:
import dask

In [ ]:
@dask.delayed
def testing():
    print('hi')

In [ ]:
testing().compute()

In [ ]:
cluster.close()